In [71]:
from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json
load_dotenv(override=True)

True

In [72]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [73]:
openai = OpenAI()

In [74]:
todos = []
completed = []

In [75]:
def get_todo_report() -> str:
    result = ""
    for index, todo in  enumerate(todos):
        if completed[index]:
            result += f"TODO #{index + 1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"TODO #{index + 1}: {todo}\n"     
    show(result)   
    return result

In [76]:
get_todo_report()

''

In [77]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [78]:
def mark_complete(index:int, completion_notes:str) -> str:
    if 1 <= index <= len(todos):
        completed[index- 1] = True
    else:
        return "No Todo at this index"
    return get_todo_report()

In [79]:
todos, completed = [], []
create_todos(["Create discipline schedule", "drink tea", "Avoid sugar"])

TODO #1: Create discipline schedule
TODO #2: drink tea
TODO #3: Avoid sugar

'TODO #1: Create discipline schedule\nTODO #2: drink tea\nTODO #3: Avoid sugar\n'

In [80]:
mark_complete(1, "Done")

TODO #1: Create discipline schedule
TODO #2: drink tea
TODO #3: Avoid sugar

'TODO #1: [green][strike]Create discipline schedule[/strike][/green]\nTODO #2: drink tea\nTODO #3: Avoid sugar\n'

In [81]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list.",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [82]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [83]:
tools =[{"type":"function", "function": create_todos_json},
        {"type":"function", "function": mark_complete_json}]

In [84]:
tools

[{'type': 'function',
  'function': {'name': 'create_todos',
   'description': 'Add new todos from a list of descriptions and return the full list.',
   'parameters': {'type': 'object',
    'properties': {'descriptions': {'type': 'array',
      'items': {'type': 'string'},
      'title': 'Descriptions'}},
    'required': ['descriptions'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'mark_complete',
   'description': 'Mark complete the todo at the given position (starting from 1) and return the full list',
   'parameters': {'properties': {'index': {'description': 'The 1-based index of the todo to mark as complete',
      'title': 'Index',
      'type': 'integer'},
     'completion_notes': {'description': 'Notes about how you completed the todo in rich console markup',
      'title': 'Completion Notes',
      'type': 'string'}},
    'required': ['index', 'completion_notes'],
    'type': 'object',
    'additionalProperties': False}}}]

In [85]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [91]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason
        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done =  True
        show(response.choices[0].message.content)


In [89]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A train leaves Boston at 2:00 pm traveling 60 mph.
Another train leaves New York at 3:00 pm traveling 80 mph toward Boston.
When do they meet?
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [92]:
todos, completed = [], []
loop(messages)

The two trains meet at approximately **4:06 pm**.